In [1]:
import pandas as pd
rating_df = pd.read_csv('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/raw/rating_AMAZON_FASHION.csv', header=None)
# rating_df = pd.read_csv('/ssd1/holdenhu/feature_dataset/Amazon_Beauty/raw/rating_All_Beauty.csv', sep='\t')
rating_df.columns = ['itemid', 'userid','rating','timestamp']
print('inter:',len(rating_df))
print('item:',len(set(rating_df['itemid'])))
print('user:',len(set(rating_df['userid'])))

inter: 883636
item: 186189
user: 749233


In [2]:
# rating<3的删除
rating_df = rating_df[~(rating_df['rating']<3)]
print('inter:',len(rating_df))
print('item:',len(set(rating_df['itemid'])))
print('user:',len(set(rating_df['userid'])))

inter: 711838
item: 162757
user: 608753


In [3]:
# 5-core
ITEM_CORE,USER_CORE = 2,2
print(len(rating_df))
del_item_num, del_user_num = 100,100

while del_item_num>0 or del_user_num>0:
    # 出现<5次的全部删除
    item_count = pd.DataFrame(rating_df['itemid'].value_counts())
    item_count.columns = ['nums']
    item_count = item_count[item_count['nums'] < ITEM_CORE]
    item_delindexs = item_count.index
    del_item_num = len(item_delindexs)
    # 出现<5次的全部删除
    user_count = pd.DataFrame(rating_df['userid'].value_counts())
    user_count.columns = ['nums']
    user_count = user_count[user_count['nums'] < USER_CORE]
    user_delindexs = user_count.index
    del_user_num = len(user_delindexs)
    if del_item_num==0 and del_user_num==0:
        print("Finish searching core.")
        break
    rating_df = rating_df[~rating_df['itemid'].isin(item_delindexs)]
    rating_df = rating_df[~rating_df['userid'].isin(user_delindexs)]
    print(len(rating_df))
print('inter:',len(rating_df))
print('item:',len(set(rating_df['itemid'])))
print('user:',len(set(rating_df['userid'])))

711838
156149
127120
115577
112499
110812
110291
110015
109929
109886
109876
109871
109870
Finish searching core.
inter: 109870
item: 13858
user: 46365


In [ ]:
# # user-based filter
# CORE=3
# print(len(rating_df))
# del_user_num = 100

# # 出现<5次的全部删除
# user_count = pd.DataFrame(rating_df['userid'].value_counts())
# user_count.columns = ['nums']
# user_count = user_count[user_count['nums'] < CORE]
# user_delindexs = user_count.index
# del_user_num = len(user_delindexs)

# rating_df = rating_df[~rating_df['userid'].isin(user_delindexs)]
# print(len(rating_df))
# print('inter:',len(rating_df))
# print('item:',len(set(rating_df['itemid'])))
# print('user:',len(set(rating_df['userid'])))

In [4]:
itemid_dict = {} # original item id => new id
userid_dict = {} # to save
all_item = list(set(rating_df['itemid']))
all_user = list(set(rating_df['userid']))
for i in all_item:
    itemid_dict[i]=len(itemid_dict)+1
for u in all_user:
    userid_dict[u]=len(userid_dict)+1

In [5]:
userid_his = list(rating_df.sort_values(['userid','timestamp'],ascending=True).groupby('userid'))

In [6]:
from tqdm import tqdm

user_hist_dict = {}
for user_record in tqdm(userid_his):
    userid = user_record[0]
    record = user_record[1]
    item_list = [itemid_dict[i] for i in list(record['itemid'])]
    t_lit = list(record['timestamp'])
    user_hist_dict[userid_dict[userid]] = [(i,t) for i,t in zip(item_list, t_lit)]
# 30364: [(12197, 1409011200), (7894, 1430524800), (11612, 1430524800)],

100%|██████████| 46365/46365 [00:05<00:00, 8787.14it/s] 


In [7]:
# train-test splition
## based on timestamp, 一刀切
split_ratio = 0.2
t_list = sorted(rating_df['timestamp'])
le = len(t_list)
split_idx = le - int(le*split_ratio)
split_time = t_list[split_idx]

all_train_seq, all_test_seq = [],[] # (u, [i0,i1,i2])
for u, record in user_hist_dict.items():
    train_data, test_data = [], []
    for i,t in record:
        if t<= split_time:
            train_data.append(i)
        else:
            test_data.append(i)
    if len(train_data)>0:
        all_train_seq.append((u,train_data))
    if len(test_data)>0:
        all_test_seq.append((u,train_data,test_data))

## leave-one-out [TODO]
## user-specific ratio-based [TODO]

In [8]:
# (3) test.txt; (4) train.txt; data[0][idx]=>sequence, data[0][idx]=>label
train = []
test = []

def process_train_seq(train_seq): # input: [u, (i0,i1,i2)]
    out_seqs = []
    labs = []
    for u,seq in train_seq: # TO: I dont use the user ID for now
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [seq[:-i]]
    return out_seqs, labs

def process_test_seq(test_seq): # input: [u, (i0,i1,i2)]
    out_seqs = []
    labs = []
    for u, train_seq, seq in test_seq: # TO: I dont use the user ID for now
        for i in range(1, len(seq)):
            tar = seq[-i]
            labs += [tar]
            out_seqs += [train_seq+seq[:-i]]
    return out_seqs, labs

tr_seqs, tr_labs = process_train_seq(all_train_seq)
te_seqs, te_labs = process_test_seq(all_test_seq)
tra = (tr_seqs, tr_labs)
tes = (te_seqs, te_labs)

In [9]:
import pickle
pickle.dump(itemid_dict, open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/itemid_dict.txt','wb'))
pickle.dump(userid_dict, open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/userid_dict.txt','wb'))
pickle.dump(all_test_seq, open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/all_test_seq.txt','wb'))
pickle.dump(all_train_seq, open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/all_train_seq.txt','wb'))

pickle.dump(tra, open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/train.txt','wb'))
pickle.dump(tes, open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/test.txt','wb'))

In [13]:
################ New thread
# train_k.txt, test_k.txt
import pickle
cluster_num=20
k=1

asin_topic_dict = pickle.load(open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/processed/text/asin_clusters{}_dict.pickle'.format(cluster_num),'rb'))
asin_image_dict = pickle.load(open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/processed/image/image_cluster{}_dict.pickle'.format(cluster_num),'rb'))
itemid_dict = pickle.load(open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/itemid_dict.txt','rb'))
train = pickle.load(open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/train.txt','rb'))
test = pickle.load(open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/test.txt','rb'))
node_num = len(itemid_dict.values())+1 # and 0<pad>
image_num = max(asin_image_dict.values())+1
text_num = max(asin_topic_dict.values())+1
print(len(itemid_dict.values()), image_num, text_num)

13858 20 20


In [14]:
reverse_item_dict = {v: k for k, v in itemid_dict.items()}

def process_vl(data):
    seqs = data[0]
    text_seqs, image_seqs = [],[]
    for seq in seqs:
        image_seq = []
        text_seq = []
        for i in seq:
            itemid = reverse_item_dict[i]
            if itemid in asin_image_dict:
                image_c = [asin_image_dict[itemid]+node_num]
            else:
                image_c = []
            image_seq.append(image_c)
            
            if itemid in asin_topic_dict:
                text_c = [asin_topic_dict[itemid]+node_num+image_num]
            else:
                text_c = []
            text_seq.append(text_c)
        text_seqs.append(text_seq)
        image_seqs.append(image_seq)
    return text_seqs, image_seqs
tra_text_seqs, tra_image_seqs = process_vl(train)
tes_text_seqs, tes_image_seqs = process_vl(test)

pickle.dump([train[0], tra_text_seqs, tra_image_seqs, train[1]], open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/train_c{}_k{}.txt'.format(cluster_num,k),'wb'))
pickle.dump([test[0], tes_text_seqs, tes_image_seqs, test[1]], open('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/test_c{}_k{}.txt'.format(cluster_num,k),'wb'))


In [1]:
import pandas as pd
_ = pd.read_csv('/ssd1/holdenhu/Amazon_dataset/Amazon_Fashion/raw/rating_AMAZON_FASHION.csv')


In [2]:
_.head()

,7106116521,A1D4G1SNUZWQOT,5.0,1413763200
0,7106116521,A3DDWDH9PX2YX2,2.0,1411862400
1,7106116521,A2MWC41EW7XL15,4.0,1408924800
2,7106116521,A2UH2QQ275NV45,2.0,1408838400
3,7106116521,A89F3LQADZBS5,3.0,1406419200
4,7106116521,A29HLOUW0NS0EH,5.0,1405728000


In [7]:
rating_df = pd.read_csv('/ssd1/holdenhu/ML_dataset/ml-1m/raw/ratings.dat',sep='::',header=None)
rating_df.columns = ['userid', 'itemid','rating','timestamp']
# UserID::MovieID::Rating::Timestamp
rating_df.head()

/home/holdenhu/miniconda3/envs/recbole/lib/python3.9/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,userid,itemid,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
